In [ ]:
from retrying import retry
import requests
import json
import hmac


class BiboxCon:

    def __init__(self, current_account, api_key, secret):
        self.current_account = current_account
        self.api_key = api_key
        self.secret = secret
        self.valid = self.connection_check()

    @retry(wait_exponential_multiplier=1000, wait_exponential_max=10000)
    def bibox_call_signed(self, uri, cmd, index, body):
        if index == "":
            command_dict = [{"cmd": cmd, "body": body}]
            print(command_dict)
        else:
            command_dict = [{"cmd": cmd, "index": index, "body": body}]

        command = json.dumps(command_dict)
        # print(command)
        signed = str(hmac.new(self.secret.encode('utf-8'), command.encode('utf-8')).hexdigest())
        params = {'cmds': command, "apikey": self.api_key, "sign": signed}

        return requests.post(url=uri, data=params).json()

    @retry(wait_exponential_multiplier=1000, wait_exponential_max=10000)
    def bibox_call(self, uri, cmd, body):
        command_dict = [{"cmd": cmd, "body": body}]
        command = json.dumps(command_dict)
        params = {'cmds': command}

        return requests.post(url=uri, data=params).json()

    @retry(wait_exponential_multiplier=1000, wait_exponential_max=10000)
    def connection_check(self):
        user_info = self.bibox_call_signed("https://api.bibox.com/v1/user", "user/userInfo", "", {})
        if "error" in user_info.keys():
            return False
        else:
            return True


In [ ]:
### GET REQUESTS

In [75]:
def get_mdpt():
    r = requests.get('https://api.bibox.com/v1/mdata?cmd=depth&pair=C20_ETH&size=2')
    depth = json.loads(r.text)['result']

    bids = depth['bids']
    asks = depth['asks']

    return (float(bids[0]['price']) + float(asks[0]['price'])) / 2

In [76]:
get_mdpt()

0.002208795

In [50]:
get_depth("C20_ETH", 20)

[{'cmd': 'user/userInfo', 'body': {}}]


KeyboardInterrupt: 

In [ ]:
### SIGNED API POST REQUESTS

In [26]:
def get_pending_orders(order_side):
    body = {"pair": ["C20_ETH"],
            "account_type": [0],
            "page": 1,
            "size": 100,
            "coin_symbol": ["C20"],
            "currency_symbol": ["ETH"],
            "order_side": [order_side]
            }

    #with open('config.json') as file:
    #    config = json.load(file)

    conn = BiboxCon('11419354', 'd2314aafde74461275e33c397b97b65b93693d94', 'a59f98178029bbdb1879e878fc83b702b87773ff')

    return conn.bibox_call_signed("https://api.bibox.com/v1/orderpending",
                                  "orderpending/orderPendingList", "12346", body)

In [107]:
def make_trades_make_money(order_type, order_side, price, amount, pay_bix = 2):
    body = {"pair": ["C20_ETH"],
            "account_type": [0],
            "order_type": [order_type],
            "order_side": [order_side],
            "pay_bix": [pay_bix],
            "price": [price],
            "amount": [1],
            }

    conn = BiboxCon('what do i do here', 'd2314aafde74461275e33c397b97b65b93693d94', 'a59f98178029bbdb1879e878fc83b702b87773ff')

    return conn.bibox_call_signed("https://api.bibox.com/v1/orderpending",
                                  "orderpending/trade", "12346", body)

In [28]:
def cancel_trade(orders_id):
    body = {"orders_id": [orders_id],
            }

    conn = BiboxCon('11419354', 'd2314aafde74461275e33c397b97b65b93693d94', 'a59f98178029bbdb1879e878fc83b702b87773ff')

    return conn.bibox_call_signed("https://api.bibox.com/v1/orderpending",
                                  "orderpending/cancelTrade", "12346", body)

In [29]:
get_pending_orders(1)

[{'cmd': 'user/userInfo', 'body': {}}]


{'result': [{'result': {'count': 1,
    'page': 1,
    'items': [{'id': 696876220,
      'createdAt': 1530177546000,
      'account_type': 0,
      'pair': 'C20_ETH',
      'coin_symbol': 'C20',
      'currency_symbol': 'ETH',
      'order_side': 1,
      'order_type': 2,
      'price': '0.00100000',
      'deal_price': '0.00000000',
      'amount': '1.0000',
      'money': '0.00100000',
      'deal_amount': '0.0000',
      'deal_percent': '0.00%',
      'deal_money': '0.00000000',
      'status': 1,
      'unexecuted': '1.0000',
      'order_from': 6}]},
   'cmd': 'orderpending/orderPendingList',
   'index': '12346'}]}

In [110]:
make_trades_make_money(2, 1, 0.0001, 1)

[{'cmd': 'user/userInfo', 'body': {}}]


{'result': [{'result': 697915679,
   'cmd': 'orderpending/trade',
   'index': '12346'}]}

In [30]:
cancel_trade(696876220)

[{'cmd': 'user/userInfo', 'body': {}}]


{'result': [{'result': '撤销中',
   'cmd': 'orderpending/cancelTrade',
   'index': '12346'}]}

In [113]:
def set_staggered_orders(max, interval, amount):

    price = get_mdpt()
    askprice = price
    bidprice = price
    
    for i in range(1,max + 1):
        make_trades_make_money(2, 1, bidprice * (1 - interval/100), amount)
        bidprice -= bidprice * interval/100
        
    for i in range(1,max + 1):
        make_trades_make_money(2, 2, askprice * (1 + interval/100), amount)
        askprice += askprice * interval/100
    

In [112]:
set_staggered_orders(5, 5, 0.01)

[{'cmd': 'user/userInfo', 'body': {}}]
[{'cmd': 'user/userInfo', 'body': {}}]
[{'cmd': 'user/userInfo', 'body': {}}]
[{'cmd': 'user/userInfo', 'body': {}}]
[{'cmd': 'user/userInfo', 'body': {}}]
